# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import g_key



### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_df = pd.read_csv("../WeatherPy/output_data/weather_df.csv")
weather_df


,Unnamed: 0,City,Lat,Long,temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,butaritari,3.0707,172.7902,82.49,65,13,11.79,KI,1628439829
1,1,mataura,-46.1927,168.8643,34.48,99,99,3.60,NZ,1628440411
2,2,saint-joseph,-21.3667,55.6167,68.31,81,19,5.99,RE,1628440412
3,3,kaitangata,-46.2817,169.8464,34.32,93,78,8.52,NZ,1628439876
4,4,chahal,15.7500,-89.5667,84.11,66,99,3.38,GT,1628440415
...,...,...,...,...,...,...,...,...,...,...
579,579,baruun-urt,46.6806,113.2792,57.85,90,91,3.76,MN,1628441193
580,580,povenets,62.8487,34.8262,57.96,95,100,7.34,RU,1628441194
581,581,novyy urengoy,66.0833,76.6333,65.88,65,13,8.37,RU,1628441196
582,582,antofagasta,-23.6500,-70.4000,60.08,64,45,12.59,CL,1628440910


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
#Configure api_key
gmaps.configure(api_key=g_key)
loc = weather_df[["Lat","Long"]]
weight = weather_df["Humidity"].astype(float)


In [4]:
# Creating a figure
figure_layout = {
    'width': '400px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(loc, weights = weight, dissipating=False, max_intensity=100, point_radius=3)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
narrowed_df = weather_df[(weather_df["temp"] < 80) & (weather_df["temp"] > 70) &
                        (weather_df["Wind Speed"] < 10) & (weather_df["Cloudiness"] == 0)].dropna()
narrowed_df = narrowed_df.reset_index()
del narrowed_df['index']
del narrowed_df['Unnamed: 0']

narrowed_df

,City,Lat,Long,temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,east london,-33.0153,27.9116,76.14,20,0,5.75,ZA,1628440514
1,paracatu,-17.2222,-46.8747,79.52,27,0,7.70,BR,1628440527
2,namwala,-15.7504,26.4384,71.37,27,0,5.37,ZM,1628440704
3,nsanje,-16.9200,35.2620,72.50,41,0,4.45,MW,1628440839
4,meadow lake,34.8014,-106.5436,79.74,12,0,7.02,US,1628440932
5,babanka,48.7097,30.4483,70.57,65,0,6.11,UA,1628441034
6,honningsvag,70.9821,25.9704,71.35,40,0,8.05,NO,1628441044
7,olhao,37.0286,-7.8411,73.29,64,0,8.05,PT,1628441170


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df = narrowed_df.loc[:,["City","Country", "Lat", "Long"]]

#Creating new column
hotel_df['Hotel Name'] = ''

#Creating the Params
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {
    "type": "lodging",
    "keyword": "hotel",
    "radius": 5000,
    "key": g_key
}


In [7]:

# use iterrows to iterate through pandas dataframe

for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    long = row["Long"]
    # get restaurant type from df
    loc = f"{lat},{long}"
    

    # add keyword to params dict
    params['location'] = loc

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {loc}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    try:
        
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
        print(f"Closest hotel is {results[0]['name']}.")

    except (KeyError, IndexError):
        print("No hotel found... skipping.")
        
    print("------------")

Retrieving Results for Index 0: -33.0153,27.9116.
Closest hotel is Southern Sun Hemingways.
------------
Retrieving Results for Index 1: -17.2222,-46.8747.
Closest hotel is Hotel Paracatu Plaza.
------------
Retrieving Results for Index 2: -15.7504,26.4384.
Closest hotel is Karibu Guest House.
------------
Retrieving Results for Index 3: -16.92,35.262.
Closest hotel is Nsanje Discovery Lodge.
------------
Retrieving Results for Index 4: 34.8014,-106.5436.
No hotel found... skipping.
------------
Retrieving Results for Index 5: 48.7097,30.4483.
No hotel found... skipping.
------------
Retrieving Results for Index 6: 70.9821,25.9704.
Closest hotel is The View.
------------
Retrieving Results for Index 7: 37.0286,-7.8411.
Closest hotel is Hotel Cidade de Olhão.
------------


In [8]:
hotel_df

,City,Country,Lat,Long,Hotel Name
0,east london,ZA,-33.0153,27.9116,Southern Sun Hemingways
1,paracatu,BR,-17.2222,-46.8747,Hotel Paracatu Plaza
2,namwala,ZM,-15.7504,26.4384,Karibu Guest House
3,nsanje,MW,-16.9200,35.2620,Nsanje Discovery Lodge
4,meadow lake,US,34.8014,-106.5436,
5,babanka,UA,48.7097,30.4483,
6,honningsvag,NO,70.9821,25.9704,The View
7,olhao,PT,37.0286,-7.8411,Hotel Cidade de Olhão


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Long"]]

In [10]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

# Add the layer to the map
fig.add_layer(markers)

# Display Map
fig


Figure(layout=FigureLayout(height='420px'))